In [ ]:
import datetime as dt
import json
from pprint import pprint

import pandas as pd
import requests

import config

In [ ]:
# pd.set_option("display.max_rows", None)
# pd.reset_option("display.max_rows")

In [ ]:
WORKSPACE_ID = config.WORKSPACE_ID
USER_ID = config.USER_ID
TOKEN_VALUE = config.TOKEN_VALUE
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

In [ ]:
headers = {
    "workspaceId": WORKSPACE_ID,
    "userId": USER_ID,
    "tokenValue": TOKEN_VALUE,
    "apiKey": API_KEY,
    "apiSecret": API_SECRET,
}

# Get location heirarchy

In [ ]:
url = "https://server.itemit.com/collections/itemit/v2.0/locations/hierarchy"

payload = {}

response = requests.request("GET", url, headers=headers, data=payload)

In [ ]:
locations = response.json()

In [ ]:
pprint(locations[0]["nodes"])

In [ ]:
location_list = []
for location in locations:
    location_list.append(location["location"]["name"])
    if location["nodes"]:
        for node in location["nodes"]:
            location_list.append(node["location"]["name"])
# location_list

In [ ]:
loc_map = {
    "*West-Forvie Building": "WFB",
    "Box Storage Room": "WFB",
    "Goods In": "WFB",
    "Overflow": "WFB",
    "Production Lines": "WFB",
    "Anne McLaren Building": "AMB",
    "CUED Pod": None,
    "Christs College": "Colleges",
    "Churchill College": "Colleges",
    "Clare College": "Colleges",
    "Clare Colony": "Colleges",
    "Clare Hall": "Colleges",
    "Corpus Christi Main Plodge": "Colleges",
    "Corpus Leckhampton": "Colleges",
    "Damaged": None,
    "Darwin College": "Colleges",
    "Downing Main Plodge": "Colleges",
    "Drop-Off Pods": "DO",
    "Engineering Pod": "DO",
    "Homerton Pod": "DO",
    "Jesus Pod": "DO",
    "Johns Pod": "DO",
    "Newnham Pod": "DO",
    "Wychfield Pod": "DO",
    "Emmanuel Main Plodge": "Colleges",
    "Fitzwilliam College": "Colleges",
    "Girton College": "Colleges",
    "Gonville & Caius College": "Colleges",
    "Homerton Main Plodge": "Colleges",
    "Homterton Pod": None,
    "Hughes Hall": "Colleges",
    "In Transit": "UMS",
    "In-Transit Paul": "UMS",
    "Jesus College": "Colleges",
    "Kings Main Plodge": "Colleges",
    "Lost": None,
    "Lucy Cavendish": "Colleges",
    "MBIT": None,
    "Magdalene College": "Colleges",
    "Murray Edwards College": "Colleges",
    "Newnham College": "Colleges",
    "Pembroke Main Plodge": "Colleges",
    "Peterhouse Main Plodge": "Colleges",
    "Queens Owlstone Croft": "Colleges",
    "Queens' College": "Colleges",
    "Ridley Hall": "Colleges",
    "Robinson College": "Colleges",
    "Selwyn College": "Colleges",
    "Sidney Sussex College": "Colleges",
    "St Catherine’s Main Plodge": "Colleges",
    "St Catherine’s St Chads": "Colleges",
    "St Edmunds College": "Colleges",
    "St Johns College": "Colleges",
    "Test Site": None,
    "Trinity College": "Colleges",
    "Trinity Hall": "Colleges",
    "Wesley House": "Colleges",
    "Westcot House": "Colleges",
    "Westfield House": "Colleges",
    "Westminster College": "Colleges",
    "Wolfson College": "Colleges",
}

assert sorted(location_list) == sorted(
    loc_map.keys()
), "Location map values do not match heirarchy from API"

## Get number of box items

In [ ]:
url = "https://server.itemit.com/items/itemit/v4/count"

filters = {
    "allOf": [
        {
            "anyOf": [
                {
                    "where": "COLLECTION",
                    "has": {"name": "Drop-Off Point Box"},
                },
                {
                    "where": "COLLECTION",
                    "has": {"name": "Delivery Box - Red"},
                },
            ]
        },
        {"allOf": []},
        {"noneOf": []},
    ]
}

payload = {
    "search": "",
    "filters": filters,
    "sorts": [],
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [ ]:
size = response.json()
assert type(size) == int, "size response should be of type int"

In [ ]:
print(size)

## Get all box items

In [ ]:
url = "https://server.itemit.com/items/itemit/v7/profiles/_search"

sorts = [{"sort": "ITEM", "by": {"name": "ASC"}}]

payload = {
    "size": size,
    "page": 1,
    "search": "",
    "filters": filters,
    "sorts": sorts,
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [ ]:
items = response.json()

In [ ]:
pprint(items[0])

In [ ]:
item = items[0]

pprint(item)

In [ ]:
print("Current locations of all Drop-Off Point Boxes\n".upper())
print(
    f"No.\t{'Box':12s}\t{'Collection':12s}\t{'Location':22s}\t{'Last Seen Time':20s}\t{'Last Seen By':20s}"
)
print(
    f"---\t{'----':12s}\t{'--------':12s}\t{'--------------':20s}\t{'------------':20s}"
)

data = {
    "box": [],
    "collection": [],
    "location": [],
    "last_seen_dt": [],
    "last_seen_by": [],
    "last_seen_by_email": [],
}

for i, item in enumerate(items):
    box = item["_source"]["name"]
    collection = [
        element["_source"]["name"]
        for element in item["parentObjects"]["elements"]
        if element["_source"]["collectionType"] == "COLLECTION"
    ]
    collection = next(iter(collection), None)

    location = [
        element["_source"]["name"]
        for element in item["parentObjects"]["elements"]
        if element["_source"]["collectionType"] == "LOCATION"
    ]
    location = next(iter(location), None)

    try:
        last_seen_time = item["_source"]["lastSeen"]["datetime"]
        last_seen_time = pd.to_datetime(last_seen_time)
    except TypeError:
        last_seen_time = None

    try:
        first_name = item["_source"]["lastSeen"]["_user"]["firstName"]
    except TypeError:
        first_name = ""
    try:
        last_name = item["_source"]["lastSeen"]["_user"]["lastName"]
    except TypeError:
        last_name = ""
    last_seen_by = f"{first_name} {last_name}"
    last_seen_by = last_seen_by if last_seen_by != " " else None

    try:
        last_seen_by_email = item["_source"]["lastSeen"]["_user"]["email"]
    except TypeError:
        last_seen_by_email = None

    data["box"].append(box)
    data["collection"].append(collection)
    data["location"].append(location)
    data["last_seen_dt"].append(last_seen_time)
    data["last_seen_by"].append(last_seen_by)
    data["last_seen_by_email"].append(last_seen_by_email)

    print(
        f"{i:2d}.\t{data['box'][i]:12.12s}\t{str(data['collection'][i]):12.12s}\t{str(data['location'][i]):22.22s}\t{str(data['last_seen_dt'][i]):20.20s}\t{str(data['last_seen_by'][i]):20.20s}"
        #         f"{i:2d}.\t{data['box'][i]:12.12s}\t{data['location'][i]:22.22s}\t{data['last_seen_dt'][i]:20.20s}\t{data['last_seen_by'][i]:20.20s}"
    )

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.sort_values("last_seen_dt", ascending=False)

In [ ]:
df["id"] = df["box"].str[:-2]
df["rota"] = df["box"].str[-1]
df

## Map actual location names to schedule location names

In [ ]:
df["loc_curr"] = df["location"]
df["loc_curr"].fillna("None")
df["loc_curr"] = df["loc_curr"].map(loc_map).fillna("Colleges")
df

# Excel Schedule

In [ ]:
df_boxA = pd.read_excel("UMS_Courier_Schedules.xlsx", sheet_name="Box_Group_Lookup")
df_boxB = df_boxA.copy()

df_boxA["rota"] = "A"
df_boxA["id"] = df_boxA["Box"]
df_boxA["Box"] = df_boxA["Box"] + "-" + df_boxA["rota"]
df_boxA["Group"] = df_boxA["Group"] + "-" + df_boxA["rota"]

df_boxB["rota"] = "B"
df_boxB["id"] = df_boxB["Box"]
df_boxB["Box"] = df_boxB["Box"] + "-" + df_boxB["rota"]
df_boxB["Group"] = df_boxB["Group"] + "-" + df_boxB["rota"]

df_box = pd.concat([df_boxA, df_boxB]).sort_values("Box").set_index("Box", drop=True)

df_box

In [ ]:
df_sch = pd.read_excel(
    "UMS_Courier_Schedules.xlsx", sheet_name="Group_Schedule", index_col="Group"
)
df_sch

In [ ]:
day = "Monday-A"
box = "HO-1-A"

group = df_box.loc[box, "Group"]


df_sch.loc[group, day]

In [ ]:
days = df_sch.columns.to_list()
boxes = df_box.index.to_list()

from typing import List


def get_expected_location(boxes: List[str], days: str) -> pd.DataFrame:
    """
    Gets expected location of boxes at the end of days

    Params
    ------
    box: List[str]
        - list of boxes
    day: str
        - day

    Returns
    -------
    pd.DataFrame

    """
    group = df_box.loc[boxes, "Group"]
    df_out = df_sch.loc[group, [days]]
    df_out["Box"] = boxes
    df_out = df_out.set_index("Box", drop=True)
    df_out = df_out.rename(columns={f"{days}": "loc_exp"})
    return df_out


# get_expected_location(box=boxes[0:2], day=days[0:2])
# get_expected_location(boxes=['HO-1-A', 'HO-1-B', 'W-1-A'], days=days[0:2])
get_expected_location(["HO-1-A"], "Sunday-B")

In [ ]:
# boxes = ['HO-1-B']
day = "Sunday-B"

df_exp = get_expected_location(boxes, day)

df_act = df.loc[df["box"].isin(boxes), ["box", "location", "loc_curr", "last_seen_dt"]]

df_locations = pd.merge(df_act, df_exp, left_on="box", right_index=True).set_index(
    "box", drop=True
)
df_locations["location_is_correct"] = (
    df_locations["loc_curr"] == df_locations["loc_exp"]
)

df_locations[~df_locations["location_is_correct"]]

In [ ]:
pd.set_option("display.max_rows", None)
df_locations.sort_values(
    ["location_is_correct", "last_seen_dt", "loc_curr", "location"]
)
# pd.reset_option("display.max_rows")